# News classification using BERT

## Import libraries

In [1]:
import math
import os

import numpy as np
import pandas as pd
import lightning as L
import torch
import torch.nn.functional as F
import torchtext.datasets as td
from lightning.pytorch.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateMonitor,
)
from lightning.pytorch.cli import LightningCLI
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchdata.datapipes.iter import IterDataPipe
from torchtext.data.functional import to_map_style_dataset
from torchtext.datasets import AG_NEWS
from transformers import BertModel, BertTokenizer, AdamW

import mlflow.pytorch

ModuleNotFoundError: No module named 'torchtext'